In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import glob
import pandas as pd
from tqdm import tqdm
os.chdir("/kaggle/input")
import nltk
import matplotlib.pyplot as plt
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
# pd.read_csv('/kaggle/input/14-million-cell-phone-reviews/phone_user_review_file_1.csv',encoding="ISO-8859-1")

In [ ]:
df = pd.DataFrame()
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in tqdm(filenames):
        x = os.path.join(dirname, filename)
        temp = pd.read_csv(x,encoding="ISO-8859-1")
#         print(type(x))
        if df.shape[0] ==0:
            df = temp
        else:
            df = pd.concat([df,temp])

In [ ]:
# df.phone_url.str.split('/')
df[df.author == 'ã¦ã©ãã©ãã©ã´ã³']

In [ ]:
df.columns

In [ ]:
#using phone url is a better way to identify the product than the product column itself
y = df['phone_url'].str.split('/',n=-1,expand = True)
df['Prod_name'] = y[2]

In [ ]:
new_df = df[['date', 'lang', 'country', 'source',  'score','extract', 'author', 'Prod_name']]

In [ ]:
new_df.shape

In [ ]:
#calculating null values across dataset
nd = {}
for i in new_df.columns:
    nd[i] = new_df[i].isnull().sum()
nd

In [ ]:
#create year out of date
x = new_df['date'].str.split('/',n=-1,expand = True)
new_df['Year'] = x[2]

Here we saw that the year starts from 1970 with only one phone from sony, which seems very unlikely and hence is an incorrect record and should be removed

In [ ]:
new_df['Year'].hist(xrot=90)
np.unique(new_df.Year)
new_df[new_df.Year=='1970']
new_df = new_df[new_df.Year!='1970']

The purpose of below code block is to find the sentiment scores for all language reviews for which we have stopwords available in NLTK
This uses a dataset with abbreviations for languages and the full forms to see if the NLTK has stopwords for those, the dataset has been removed for the purpose of restarting the notebook and keeping the results as it is
Also we are calculating which reviewers among these rows with non nulls have amazon in their title- thereby calculating their trends across years and across the source where the reviews are published

In [ ]:
lang = pd.read_csv("../input/language-codes/language-codes.csv")
lang.columns = ['lang','actual']
new_df2 = pd.merge(new_df,lang, on='lang')
#lowercasing the reviews and languages acronyms
new_df2['extract'] = new_df2['extract'].astype(str)
new_df2['actual'] = new_df2['actual'].astype(str)
new_df2['extract'] = new_df2['extract'].apply(lambda x: " ".join(x.lower() for x in x.split()))
new_df2['actual'] = new_df2['actual'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#removing extract stopwords from all available languages in stopwords list
lang_av = ['arabic',
 'chinese',
 'czech',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hebrew',
 'hungarian',
 'indonesian',
 'italian',
 'japanese',
 'korean',
 'norwegian',
 'polish',
 'portuguese',
 'russian',
 'spanish',
 'swedish',
 'turkish']
# print(stopwords.fileids())
final_df = pd.DataFrame()
for i in lang_av:
    try:
        stop = stopwords.words(i)
        temp = new_df2[new_df2.actual == i]
        temp['extract'] = temp['extract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
        if final_df.shape[0]==0:
            final_df = temp
        else:
            final_df = pd.concat([final_df,temp])
    except:
        pass
#calculating sentiment score using inbuilt function textblob
from textblob import TextBlob
def senti(x):
    return TextBlob(x).sentiment
final_df_copy=  final_df.copy(deep = True)
final_df_copy['senti_score'] = final_df_copy['extract'].apply(senti)
#getting rid of null values only to see the anonymous reviews
final_df_copy_nonNull = final_df_copy.dropna(subset = ['author'])
#the number of reviews given by people with just amazon title
final_df_copy_nonNull[final_df_copy_nonNull['author'].str.match('Amazon')]

In [ ]:
final_df

In [ ]:
final_df_copy.senti_score.value_counts()

In [ ]:
#the reviews with amazon heading vary across different countries in following way
final_df_copy_nonNull[final_df_copy_nonNull['author'].str.match('Amazon')].country.hist()

In [ ]:
#the number of people reviewing with amazon customer as their title has increased significantly in the last 3 years
final_df_copy_nonNull[final_df_copy_nonNull['author'].str.match('Amazon')].Year.hist(xrot = 90)

This code block creates a visual to see how across all null or non-null reviewers' name, the source has been used across years to write reviews

In [ ]:
source_dropdown = widgets.Dropdown(options = list(np.unique(new_df.source)),value= None,description='Select source')
def source_variation_across_years(x):
    selected_source = source_dropdown.value
    selected_df = new_df[new_df.source == selected_source]
    selected_df.Year.hist(xrot = 90)
interact(source_variation_across_years,x =source_dropdown )

In [ ]:
#creating a new column to find the company scores across years
z = new_df['Prod_name'].str.split('-',n=-1,expand=True)
new_df['Company'] = z[0]
new_df

In [ ]:
companyNnumbers20 = pd.DataFrame(new_df.Company.value_counts()).reset_index().iloc[:20,]
companyNnumbers20.columns = ['CompanyName','Reviews']

In [ ]:
companyNnumbers20

The below code block shows how the trend of top 20 companies in terms of number of reviews had their trend changed across years across review counts

In [ ]:
company_seletion = widgets.Dropdown(options = list(np.unique(companyNnumbers20.CompanyName)),value= None,description='Select company')
def company_reviews_across_years(x):
    selected_company = company_seletion.value
    selected_df = new_df[new_df.Company == selected_company]
    selected_df.sort_values(by='Year').Year.hist(xrot = 90)
interact(company_reviews_across_years,x =company_seletion )

In [ ]:
new_df.score

In [ ]:
filled_df = new_df.fillna('ThisOne')
dict(filled_df[filled_df.score == 'ThisOne'].Company.value_counts())

In [ ]:
#we can infer here that 5 countries make up 52000 of 63000 null scores
countries_with_nullscores = filled_df[filled_df.score=='ThisOne'].country.value_counts()
countries_with_nullscores

In [ ]:
#these countries have been chosen because of their null score to total count ratio
countries_to_remove = ['tr','fr','br']
for i in countries_to_remove:
    ratio = countries_with_nullscores[i]/len(filled_df[filled_df.country==i])
    print(f'null to total ratio for {i} is {ratio}')

In [ ]:
#this dataframe is devoid of countries with high null score to total ratio
null_score_calc_df = new_df[~new_df.country.isin(countries_to_remove)]
null_score_calc_df

In [ ]:
#we can also calculate which coutries have english speaking reviewers
new_df.country[new_df.lang == 'en'].value_counts()

In [ ]:
new_df.score.fillna(5.0)

In [ ]:
new_df.score.value_counts()

In [ ]:
#now we calculate the average score for each company across the years and visualize how these scores have varied
new_df.head()

In [ ]:
fdd = pd.DataFrame()
for i in tqdm(companyNnumbers20.CompanyName):
    temp = np.unique(new_df.Year[new_df.Company==i])
    for j in tqdm(temp):
        mean_score = np.mean(new_df[(new_df.Company == i) & (new_df.Year == j)].score)
        temp_df = pd.DataFrame([i,j,mean_score]).T
        if fdd.shape[0]==0:
            fdd = temp_df
        else:
            fdd = pd.concat([fdd,temp_df])
fdd

In [ ]:
fdd.columns = ['Company','Year','MeanScore']
fdd.Year = fdd.Year.astype(int)

In [ ]:
company_seletion = widgets.Dropdown(options = list(np.unique(companyNnumbers20.CompanyName)),value= 'apple',description='Select company')
def company_scores_across_years(t):
    selected_company = company_seletion.value
    selected_df = fdd[fdd.Company == selected_company]
    year_list = list(np.unique(selected_df.Year))
#     return selected_df
    selected_df.plot.line(x='Year',y='MeanScore',rot=90,marker='o')
    print(year_list)
    plt.xticks(year_list)
interact(company_scores_across_years,t =company_seletion )

In [ ]:
new_df[(new_df.Year == '2000')&(new_df.Company=='samsung')].Prod_name.value_counts()

In [ ]:
new_df[(new_df.Year == '2008')&(new_df.Company=='samsung')].Prod_name.value_counts()

In [ ]:
new_df[(new_df.Year == '1999')&(new_df.Company=='samsung')].Prod_name.value_counts()

In [ ]:
new_df[(new_df.Year == '2016')&(new_df.Company=='samsung')].Prod_name.value_counts()

In [ ]:
#adding extra information to identify the products which plummeted the score in that year